In [ ]:
# 라이브러리 임포트
import os
import pandas as pd

from framework.dataset_specification import NamedDatasetSpecifications
from framework.enumerations import EvaluationDatasetSampling
from framework.flow_transformer import FlowTransformer
from framework.flow_transformer_parameters import FlowTransformerParameters
from framework.framework_component import FunctionalComponent
from implementations.classification_heads import *
from implementations.input_encodings import *
from implementations.pre_processings import StandardPreProcessing
from implementations.transformers.basic_transformers import BasicTransformer
from implementations.transformers.named_transformers import *


In [ ]:
# ========== 사용자 설정 ==========
# CICFlowMeter로 생성한 CSV 파일 경로
dataset_path = r"C:\Dataset\cic2017\DoS-Wednesday-WorkingHours.pcap_ISCX.csv"

# 캐시 폴더 (선택사항 - 처리 속도 향상)
cache_folder = r"C:\Dataset\cache"

# 학습 설정
epochs = 10
batch_size = 128
evaluation_percent = 0.2  # 20%를 평가용으로 사용
# ================================

print(f"데이터셋 경로: {dataset_path}")
print(f"캐시 폴더: {cache_folder}")
print(f"학습 에포크: {epochs}")
print(f"배치 크기: {batch_size}")
print(f"평가 데이터 비율: {evaluation_percent*100}%")


In [ ]:
# 다양한 인코딩 옵션
encodings = [
    NoInputEncoder(),                                    # [0] 기본 인코더
    RecordLevelEmbed(64),                               # [1] 레코드 레벨 임베딩
    CategoricalFeatureEmbed(EmbedLayerType.Dense, 16),  # [2] Dense 임베딩
    CategoricalFeatureEmbed(EmbedLayerType.Lookup, 16), # [3] Lookup 임베딩
    CategoricalFeatureEmbed(EmbedLayerType.Projection, 16), # [4] Projection 임베딩
    RecordLevelEmbed(64, project=True)                  # [5] 프로젝션 포함 레코드 임베딩
]

# 다양한 분류 헤드 옵션
classification_heads = [
    LastTokenClassificationHead(),           # [0] 마지막 토큰 기반
    FlattenClassificationHead(),            # [1] 플래튼 기반
    GlobalAveragePoolingClassificationHead(), # [2] 글로벌 평균 풀링
    CLSTokenClassificationHead(),           # [3] CLS 토큰 기반
    FeaturewiseEmbedding(project=False),    # [4] 피처별 임베딩
    FeaturewiseEmbedding(project=True),     # [5] 프로젝션 포함 피처별 임베딩
]

# 다양한 트랜스포머 옵션
transformers = [
    BasicTransformer(2, 128, n_heads=2),                    # [0] 기본 인코더
    BasicTransformer(2, 128, n_heads=2, is_decoder=True),   # [1] 기본 디코더
    GPTSmallTransformer(),                                  # [2] GPT 스타일
    BERTSmallTransformer()                                  # [3] BERT 스타일
]

print(f"인코딩 옵션: {len(encodings)}개")
print(f"분류 헤드 옵션: {len(classification_heads)}개")
print(f"트랜스포머 옵션: {len(transformers)}개")


In [ ]:
# 전처리 설정
pre_processing = StandardPreProcessing(n_categorical_levels=32)

# FlowTransformer 정의
# 기본 설정: NoInputEncoder + BasicTransformer + LastTokenClassificationHead
ft = FlowTransformer(
    pre_processing=pre_processing,
    input_encoding=encodings[0],        # 다른 인코더를 사용하려면 인덱스 변경
    sequential_model=transformers[0],   # 다른 트랜스포머를 사용하려면 인덱스 변경
    classification_head=classification_heads[0],  # 다른 분류 헤드를 사용하려면 인덱스 변경
    params=FlowTransformerParameters(
        window_size=8,              # 시퀀스 윈도우 크기
        mlp_layer_sizes=[128],      # MLP 레이어 크기
        mlp_dropout=0.1             # 드롭아웃 비율
    )
)

print("FlowTransformer 초기화 완료")
print(f"사용 중인 인코더: {type(ft.input_encoding).__name__}")
print(f"사용 중인 트랜스포머: {type(ft.sequential_model).__name__}")
print(f"사용 중인 분류 헤드: {type(ft.classification_head).__name__}")


In [ ]:
## 샘플 데이터 생성 (실제 CSV 파일이 없는 경우)

import numpy as np

# 샘플 데이터 생성 함수
def create_sample_cicflowmeter_data(file_path, num_samples=1000):
    """CICFlowMeter 형식의 샘플 데이터 생성"""
    np.random.seed(42)  # 재현 가능한 결과를 위해
    
    # CICFlowMeter V3 스펙에 맞는 컬럼들
    columns = [
        'Flow Duration', 'Total Fwd Packets', 'Total Backward Packets',
        'Total Length of Fwd Packets', 'Total Length of Bwd Packets',
        'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Mean',
        'Fwd Packet Length Std', 'Bwd Packet Length Max', 'Bwd Packet Length Min',
        'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s',
        'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
        'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std',
        'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean',
        'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags',
        'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
        'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
        'Min Packet Length', 'Max Packet Length', 'Packet Length Mean',
        'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count',
        'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count',
        'URG Flag Count', 'CWE Flag Count', 'ECE Flag Count', 'Down/Up Ratio',
        'Average Packet Size', 'Avg Fwd Segment Size', 'Avg Bwd Segment Size',
        'Fwd Header Length.1', 'Fwd Avg Bytes/Bulk', 'Fwd Avg Packets/Bulk',
        'Fwd Avg Bulk Rate', 'Bwd Avg Bytes/Bulk', 'Bwd Avg Packets/Bulk',
        'Bwd Avg Bulk Rate', 'Subflow Fwd Packets', 'Subflow Fwd Bytes',
        'Subflow Bwd Packets', 'Subflow Bwd Bytes', 'Init_Win_bytes_forward',
        'Init_Win_bytes_backward', 'act_data_pkt_fwd', 'min_seg_size_forward',
        'Active Mean', 'Active Std', 'Active Max', 'Active Min',
        'Idle Mean', 'Idle Std', 'Idle Max', 'Idle Min', 'Label'
    ]
    
    # 샘플 데이터 생성
    data = {}
    for col in columns[:-1]:  # Label 제외
        if 'Flag' in col:
            # 플래그 컬럼은 0-10 범위의 정수
            data[col] = np.random.randint(0, 11, num_samples)
        elif 'Packet' in col and 'Length' in col:
            # 패킷 길이 관련은 양수 값
            data[col] = np.random.exponential(500, num_samples)
        elif 'Duration' in col or 'IAT' in col:
            # 시간 관련은 마이크로초 단위
            data[col] = np.random.exponential(1000000, num_samples)
        elif 'Bytes' in col:
            # 바이트 관련은 양수 값
            data[col] = np.random.exponential(1000, num_samples)
        else:
            # 기타 피처들
            data[col] = np.random.normal(100, 50, num_samples)
    
    # Label 컬럼 (이진 분류)
    data['Label'] = np.random.choice(['BENIGN', 'ATTACK'], num_samples, p=[0.8, 0.2])
    
    # DataFrame 생성 및 저장
    df = pd.DataFrame(data)
    
    # 디렉토리가 없으면 생성
    os.makedirs(os.path.dirname(file_path), exist_ok=True)
    
    df.to_csv(file_path, index=False)
    print(f"샘플 데이터 생성 완료: {file_path}")
    print(f"행 수: {len(df)}, 열 수: {len(df.columns)}")
    print(f"레이블 분포: {df['Label'].value_counts().to_dict()}")
    
    return file_path

# 실제 파일이 없으면 샘플 데이터 생성
if not os.path.exists(dataset_path):
    print(f"실제 데이터셋 파일이 없으므로 샘플 데이터를 생성합니다...")
    dataset_path = create_sample_cicflowmeter_data(dataset_path, num_samples=5000)
else:
    print(f"기존 데이터셋 파일 사용: {dataset_path}")


In [ ]:
# 파일 존재 확인
if not os.path.exists(dataset_path):
    print(f"오류: 데이터셋 파일을 찾을 수 없습니다: {dataset_path}")
    print("위의 셀에서 dataset_path를 올바른 경로로 설정해주세요.")
else:
    print(f"데이터셋 파일 확인됨: {dataset_path}")
    
    # 데이터셋 로드 설정
    dataset_name = "cic2017_Dos_dataset"
    
    dataset_specification = NamedDatasetSpecifications.cic_2017_dataset
    eval_method = EvaluationDatasetSampling.RandomRows
    
    print("\n데이터셋 로딩 시작...")
    print(f"데이터셋 스펙: {dataset_specification.class_column} 컬럼을 클래스로 사용")
    print(f"포함된 피처 수: {len(dataset_specification.include_fields)}개")
    print(f"범주형 피처 수: {len(dataset_specification.categorical_fields)}개")
    
    # 캐시 폴더 사용 여부에 따라 다르게 로드
    if cache_folder and os.path.exists(os.path.dirname(cache_folder)):
        ft.load_dataset(
            dataset_name, 
            dataset_path, 
            dataset_specification, 
            cache_folder,
            evaluation_dataset_sampling=eval_method, 
            evaluation_percent=evaluation_percent
        )
        print(f"캐시 폴더 사용: {cache_folder}")
    else:
        ft.load_dataset(
            dataset_name, 
            dataset_path, 
            dataset_specification, 
            evaluation_dataset_sampling=eval_method, 
            evaluation_percent=evaluation_percent
        )
        print("캐시 없이 로드")
    
    print("데이터셋 로딩 완료!")


In [ ]:
print("모델 빌드 중...")
model = ft.build_model()

print("\n=== 모델 구조 ===")
model.summary()

print("\n모델 컴파일 중...")
model.compile(
    optimizer="adam", 
    loss='binary_crossentropy', 
    metrics=['binary_accuracy'], 
    jit_compile=True
)

print("모델 준비 완료!")


In [ ]:
print("모델 빌드 중...")
model = ft.build_model()

print("\n=== 모델 구조 ===")
model.summary()

print("\n모델 컴파일 중...")
model.compile(
    optimizer="adam", 
    loss='binary_crossentropy', 
    metrics=['binary_accuracy'], 
    jit_compile=True
)

print("모델 준비 완료!")


In [ ]:
print("모델 빌드 중...")
model = ft.build_model()

print("\n=== 모델 구조 ===")
model.summary()

print("\n모델 컴파일 중...")
model.compile(
    optimizer="adam", 
    loss='binary_crossentropy', 
    metrics=['binary_accuracy'], 
    jit_compile=True
)

print("모델 준비 완료!")


In [ ]:
print("모델 빌드 중...")
model = ft.build_model()

print("\n=== 모델 구조 ===")
model.summary()

print("\n모델 컴파일 중...")
model.compile(
    optimizer="adam", 
    loss='binary_crossentropy', 
    metrics=['binary_accuracy'], 
    jit_compile=True
)

print("모델 준비 완료!")


In [ ]:
print("모델 빌드 중...")
model = ft.build_model()

print("\n=== 모델 구조 ===")
model.summary()

print("\n모델 컴파일 중...")
model.compile(
    optimizer="adam", 
    loss='binary_crossentropy', 
    metrics=['binary_accuracy'], 
    jit_compile=True
)

print("모델 준비 완료!")


In [ ]:
print("모델 빌드 중...")
model = ft.build_model()

print("\n=== 모델 구조 ===")
model.summary()

print("\n모델 컴파일 중...")
model.compile(
    optimizer="adam", 
    loss='binary_crossentropy', 
    metrics=['binary_accuracy'], 
    jit_compile=True
)

print("모델 준비 완료!")


In [ ]:
print("모델 빌드 중...")
model = ft.build_model()

print("\n=== 모델 구조 ===")
model.summary()

print("\n모델 컴파일 중...")
model.compile(
    optimizer="adam", 
    loss='binary_crossentropy', 
    metrics=['binary_accuracy'], 
    jit_compile=True
)

print("모델 준비 완료!")


In [ ]:
print("모델 빌드 중...")
model = ft.build_model()

print("\n=== 모델 구조 ===")
model.summary()

print("\n모델 컴파일 중...")
model.compile(
    optimizer="adam", 
    loss='binary_crossentropy', 
    metrics=['binary_accuracy'], 
    jit_compile=True
)

print("모델 준비 완료!")


In [ ]:
print("모델 빌드 중...")
model = ft.build_model()

print("\n=== 모델 구조 ===")
model.summary()

print("\n모델 컴파일 중...")
model.compile(
    optimizer="adam", 
    loss='binary_crossentropy', 
    metrics=['binary_accuracy'], 
    jit_compile=True
)

print("모델 준비 완료!")


In [ ]:
print("모델 빌드 중...")
model = ft.build_model()

print("\n=== 모델 구조 ===")
model.summary()

print("\n모델 컴파일 중...")
model.compile(
    optimizer="adam", 
    loss='binary_crossentropy', 
    metrics=['binary_accuracy'], 
    jit_compile=True
)

print("모델 준비 완료!")


In [ ]:
print("모델 빌드 중...")
model = ft.build_model()

print("\n=== 모델 구조 ===")
model.summary()

print("\n모델 컴파일 중...")
model.compile(
    optimizer="adam", 
    loss='binary_crossentropy', 
    metrics=['binary_accuracy'], 
    jit_compile=True
)

print("모델 준비 완료!")


In [ ]:
print("모델 빌드 중...")
model = ft.build_model()

print("\n=== 모델 구조 ===")
model.summary()

print("\n모델 컴파일 중...")
model.compile(
    optimizer="adam", 
    loss='binary_crossentropy', 
    metrics=['binary_accuracy'], 
    jit_compile=True
)

print("모델 준비 완료!")


In [ ]:
print("모델 빌드 중...")
model = ft.build_model()

print("\n=== 모델 구조 ===")
model.summary()

print("\n모델 컴파일 중...")
model.compile(
    optimizer="adam", 
    loss='binary_crossentropy', 
    metrics=['binary_accuracy'], 
    jit_compile=True
)

print("모델 준비 완료!")


In [ ]:
print("모델 빌드 중...")
model = ft.build_model()

print("\n=== 모델 구조 ===")
model.summary()

print("\n모델 컴파일 중...")
model.compile(
    optimizer="adam", 
    loss='binary_crossentropy', 
    metrics=['binary_accuracy'], 
    jit_compile=True
)

print("모델 준비 완료!")


In [ ]:
print("모델 빌드 중...")
model = ft.build_model()

print("\n=== 모델 구조 ===")
model.summary()

print("\n모델 컴파일 중...")
model.compile(
    optimizer="adam", 
    loss='binary_crossentropy', 
    metrics=['binary_accuracy'], 
    jit_compile=True
)

print("모델 준비 완료!")


In [ ]:
print("모델 빌드 중...")
model = ft.build_model()

print("\n=== 모델 구조 ===")
model.summary()

print("\n모델 컴파일 중...")
model.compile(
    optimizer="adam", 
    loss='binary_crossentropy', 
    metrics=['binary_accuracy'], 
    jit_compile=True
)

print("모델 준비 완료!")


In [ ]:
print("모델 빌드 중...")
model = ft.build_model()

print("\n=== 모델 구조 ===")
model.summary()

print("\n모델 컴파일 중...")
model.compile(
    optimizer="adam", 
    loss='binary_crossentropy', 
    metrics=['binary_accuracy'], 
    jit_compile=True
)

print("모델 준비 완료!")


In [ ]:
print("모델 빌드 중...")
model = ft.build_model()

print("\n=== 모델 구조 ===")
model.summary()

print("\n모델 컴파일 중...")
model.compile(
    optimizer="adam", 
    loss='binary_crossentropy', 
    metrics=['binary_accuracy'], 
    jit_compile=True
)

print("모델 준비 완료!")


In [ ]:
print("모델 빌드 중...")
model = ft.build_model()

print("\n=== 모델 구조 ===")
model.summary()

print("\n모델 컴파일 중...")
model.compile(
    optimizer="adam", 
    loss='binary_crossentropy', 
    metrics=['binary_accuracy'], 
    jit_compile=True
)

print("모델 준비 완료!")


In [ ]:
print("모델 빌드 중...")
model = ft.build_model()

print("\n=== 모델 구조 ===")
model.summary()

print("\n모델 컴파일 중...")
model.compile(
    optimizer="adam", 
    loss='binary_crossentropy', 
    metrics=['binary_accuracy'], 
    jit_compile=True
)

print("모델 준비 완료!")


In [ ]:
print(f"모델 학습 시작 (에포크: {epochs}, 배치 크기: {batch_size})...")
print("학습 중에는 진행 상황이 표시됩니다.\n")

# 학습 및 평가 실행
train_results, eval_results, final_epoch = ft.evaluate(
    model, 
    batch_size=batch_size, 
    epochs=epochs,
    steps_per_epoch=64, 
    early_stopping_patience=5
)

print("\n" + "="*50)
print("학습 완료!")
print(f"최종 에포크: {final_epoch}")
print("="*50)


In [ ]:
print("모델 빌드 중...")
model = ft.build_model()

print("\n=== 모델 구조 ===")
model.summary()

print("\n모델 컴파일 중...")
model.compile(
    optimizer="adam", 
    loss='binary_crossentropy', 
    metrics=['binary_accuracy'], 
    jit_compile=True
)

print("모델 준비 완료!")


In [ ]:
print("모델 빌드 중...")
model = ft.build_model()

print("\n=== 모델 구조 ===")
model.summary()

print("\n모델 컴파일 중...")
model.compile(
    optimizer="adam", 
    loss='binary_crossentropy', 
    metrics=['binary_accuracy'], 
    jit_compile=True
)

print("모델 준비 완료!")


In [ ]:
print("모델 빌드 중...")
model = ft.build_model()

print("\n=== 모델 구조 ===")
model.summary()

print("\n모델 컴파일 중...")
model.compile(
    optimizer="adam", 
    loss='binary_crossentropy', 
    metrics=['binary_accuracy'], 
    jit_compile=True
)

print("모델 준비 완료!")


In [ ]:
print("모델 빌드 중...")
model = ft.build_model()

print("\n=== 모델 구조 ===")
model.summary()

print("\n모델 컴파일 중...")
model.compile(
    optimizer="adam", 
    loss='binary_crossentropy', 
    metrics=['binary_accuracy'], 
    jit_compile=True
)

print("모델 준비 완료!")


In [ ]:
print("모델 빌드 중...")
model = ft.build_model()

print("\n=== 모델 구조 ===")
model.summary()

print("\n모델 컴파일 중...")
model.compile(
    optimizer="adam", 
    loss='binary_crossentropy', 
    metrics=['binary_accuracy'], 
    jit_compile=True
)

print("모델 준비 완료!")


In [ ]:
print("=== 최종 평가 결과 ===")
print(eval_results)
print("\n=== 학습 히스토리 ===")
print(train_results)

# 결과를 DataFrame으로 저장
if isinstance(eval_results, pd.DataFrame) and not eval_results.empty:
    print("\n=== 성능 요약 ===")
    if 'binary_accuracy' in eval_results.columns:
        accuracy = eval_results['binary_accuracy'].iloc[-1]
        print(f"최종 정확도: {accuracy:.4f} ({accuracy*100:.2f}%)")
    
    if 'loss' in eval_results.columns:
        loss = eval_results['loss'].iloc[-1]
        print(f"최종 손실: {loss:.4f}")
else:
    print("평가 결과를 분석할 수 없습니다.")


In [ ]:
print("모델 빌드 중...")
model = ft.build_model()

print("\n=== 모델 구조 ===")
model.summary()

print("\n모델 컴파일 중...")
model.compile(
    optimizer="adam", 
    loss='binary_crossentropy', 
    metrics=['binary_accuracy'], 
    jit_compile=True
)

print("모델 준비 완료!")


In [ ]:
# 학습된 모델을 저장하려면 아래 주석을 해제하고 경로를 설정하세요

model_save_path = "cicflowmeter_flowtransformer_model.h5"
model.save(model_save_path)
print(f"모델이 저장되었습니다: {model_save_path}")

# 평가 결과도 저장
eval_results.to_csv("evaluation_results.csv", index=False)
print("평가 결과가 저장되었습니다: evaluation_results.csv")

print("저장을 원하면 위의 주석을 해제하고 실행하세요.")


In [ ]:
print("모델 빌드 중...")
model = ft.build_model()

print("\n=== 모델 구조 ===")
model.summary()

print("\n모델 컴파일 중...")
model.compile(
    optimizer="adam", 
    loss='binary_crossentropy', 
    metrics=['binary_accuracy'], 
    jit_compile=True
)

print("모델 준비 완료!")


In [ ]:
print("모델 빌드 중...")
model = ft.build_model()

print("\n=== 모델 구조 ===")
model.summary()

print("\n모델 컴파일 중...")
model.compile(
    optimizer="adam", 
    loss='binary_crossentropy', 
    metrics=['binary_accuracy'], 
    jit_compile=True
)

print("모델 준비 완료!")
